In [1]:
import sys

sys.path.append("..")

from src.ssunet.dataloader import BinomDataset, ValidationDataset
from src.ssunet import SSUnet, load_config

config = load_config("config/config.yml")
data = config.path_config.load_ssunet_data()
data.binxy(mode="max")

model = SSUnet(config.model_config)
train_data = BinomDataset(data, config.data_config, split_params=config.split_params)
validation_data = ValidationDataset
train_loader = config.loader_config.loader(train_data)
trainer = config.train_config.trainer
print(f"input_size: {tuple(next(iter(train_loader))[1].shape)}")

DirectoryNotFoundError: Directory ..\tempdata\Simulated_data does not exist


DirectoryNotFoundError: Directory ..\tempdata\Simulated_data does not exist

In [ ]:
trainer.logger

In [2]:
trainer.fit(model, train_loader, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | down_convs | ModuleList | 25.0 M
1 | up_convs   | ModuleList | 10.0 M
2 | conv_final | Sequential | 33    
------------------------------------------
34.9 M    Trainable params
0         Non-trainable params
34.9 M    Total params
139.786   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 960.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.03 GiB is allocated by PyTorch, and 14.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)